## `Test1` : Retrieve Stock Data

In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import pickle
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:

# Get A list of S&P 500 tickers from a Wikipedia page
sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'


def get_sp500_tickers(url=sp500_url):
    try:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'lxml')
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)  # Exit with error message

    table = soup.find('table', {'class': 'wikitable'})
    rows = table.find_all('tr')[1:]  # row data for details of each company
    tickers = []
    for row in rows:
        data = row.find_all('td')
        tickers.append(data[0].text.strip())

    # Save the tickers in a pickle file so it does not load all the time
    pickle.dump(tickers, open('data/sp500.p', 'wb'))

    return tickers


# Run only once to save a pickle file
get_sp500_tickers()


In [ ]:
def get_yahoo_stock_data(reload_sp500_data=False):
    
    if reload_sp500_data:
        tickers = get_sp500_companies()
    else:
        tickers = pickle.load(open('data/sp500.p', 'rb'))
    
    # Get stock data with rearranged tickers
    tickers = ' '.join(tickers)        
    data = yf.download(tickers=tickers, period='max', interval='1d')
    
    # Save data 
    data['Adj Close'].to_csv('adj_prices.csv')
    data['Close'].to_csv('prices.csv')
    data['Volume'].to_csv('volumes.csv')
        

## `Test 2`: Inpecting the train data

In [ ]:
features = ['AGE', 'EDCL', 'MARRIED', 'KIDS', 'OCCAT1', 'WSAVED', 'NETWORTH', 'INCOME', 'RISK']
usecols = [f+'07' for f in features]

In [ ]:
dataset = pd.read_excel('../StockData/SCFP2009panel.xlsx', engine='openpyxl', sheet_name='SCFP2009panel', 
                        header=0, usecols=usecols)

In [ ]:
dataset.head()

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
dataset.describe()

In [ ]:
dataset['NETWORTH07'].sort_values()

---
## `Test 3`: Visualizaing a dashboard with Dash

In [1]:
# Load Dash packages
# import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc # use bootstrap
from dash.dependencies import Input, Output, State

In [2]:
# Load the other packages needed
import pandas as pd
import numpy as np
import pickle

In [3]:
# Solver
import cvxopt as opt
from cvxopt import blas, solvers

### Configure a dashboard app 

Using Bootstrap

In [4]:
# Configuration using Bootstrap theme 
# Sandstone theme - https://bootswatch.com/sandstone/

# app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SANDSTONE]) 
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SANDSTONE])

### Create a form component

In [5]:
# Create a form component for x inputs

# Feature 1: AGE
age_input = dbc.FormGroup(
    [
        dbc.Label('Select your age group', html_for='slider'),
        dcc.Slider(
            id='age-input', 
            min=10, 
            max=100, 
            step=10, 
            value=30,
            marks={ i: str(i) for i in range(10, 100+10, 10) }
        ),
        dbc.FormText("For example, if you are 20 - 29, select 20"),    
    ]
)


# Feature 2: EDUCATION
education_input = dbc.FormGroup(
    [
        dbc.Label('Select your education group', html_for='dropdown'),
        dcc.Dropdown(
            id='education-input',
            options=[
                {'label': '1: No High School Degree', 'value':1},
                {'label': '2: High School Degree','value':2}, 
                {'label': '3: Associate Degree','value':3},
                {'label': '4: College Degree','value':4}
            ],
            value=1
        ),
        dbc.FormText("For example, if post graduate falls into 4: College Degree"),    
    ]
)

# Feature 3: MARRIED
married_input = dbc.FormGroup(
    [
        dbc.Label('Select your marital status'),
        dbc.RadioItems(
            id='married-input',
            options=[
                {'label': 'Married', 'value':1},
                {'label': 'Not Married','value':2}, 
            ],
            value=1
        ),
        dbc.FormText("'Not Married' includes all marital status besides being married." 
                     "For example, it includes seperated, divorced, etc."),    
    ]
)

# Feature 4: KIDS
kids_input = dbc.FormGroup(
    [
        dbc.Label('Select mumber of your kids'),
        dbc.Input(
            id='kids-input',
            type='number',
            min=0,
            step=1,
            value=0
        ),
    ]
)

# Feature 5: OCCUPATION
occupation_input = dbc.FormGroup(
    [
        dbc.Label('Select the level of your occupation', html_for='dropdown'),
        dcc.Dropdown(
            id='occupation-input',
            options=[
                {'label': '1: Managerial', 'value':1},
                {'label': '2: Supervisory','value':2}, 
                {'label': '3: Entry-level / Associate','value':3},
                {'label': '4: Unemployed','value':4}
            ],
            value=1
        ),
        dbc.FormText("Part-time employees are considered as employeed"),    
    ]
)

# Feature 6: WSAVE - SPENDING HABIT
wsave_input = dbc.FormGroup(
    [
        dbc.Label('Select an option that is closest to your spending habit', html_for='dropdown'),
        dbc.FormText("Hint: Reflect on your habit last 6 months"),    
        dcc.Dropdown(
            id='wsave-input',
            options=[
                {'label': '1: I spent more than my income', 'value': 1},
                {'label': '2: I spent similar to my income', 'value': 2},
                {'label': '3: I spent less than my income', 'value': 3},
            ]
        )
    ]
)

# Feature 7: NETWORTH
networth_input = dbc.FormGroup(
    [
        dbc.Label('Select your net worth', html_for='slider'),
        dcc.Slider(
            id='networth-input',

            # Arbitrary numbers used here
            min = -1000000, 
            max = 3000000,
            marks = {
                -1000000: 'Below-$1M',
                0: '$0',
                1000000: '$1M',
                2000000: '$2M',
                3000000: 'Over $3M'
            },                
            value = 10000),
    ]
) 

# Feature 8: INCOME
income_input = dbc.FormGroup(
    [
        dbc.Label('Select your income', html_for='slider'),
        dcc.Slider(
            id='income-input',

            # Arbitrary numbers used here
            min = -1000000, 
            max = 3000000,
            marks = {
                -1000000: 'Below-$1M',
                0: '$0',
                1000000: '$1M',
                2000000: '$2M',
                3000000: 'Over $3M'
            },                
            value = 10000),
    ]
) 

# Feature 9: WILLINGNESS TO TAKE RISK
risk_input = dbc.FormGroup(
    [
        dbc.Label('How much are you willing to take risk in general?', html_for='dropdown'),
        dcc.Dropdown(
            id='risk-input',
            options=[
                {'label': '1: Very likely', 'value':1},
                {'label': '2: Somewhat likely','value':2}, 
                {'label': '3: Unlikely','value':3},
                {'label': '4: Never','value':4}
            ],
            value=1
        ),
    ]
) 


# Button to submit the inputs
button_input_submit = dbc.Button(id='button-risktolerance', 
                                 children='Calculate my risk tolerance',
                                 color="primary", 
                                 block=True)

# Create a form component

form = dbc.Form(
    [
        age_input, education_input, married_input, kids_input, occupation_input, 
        wsave_input, networth_input, income_input, risk_input,
        button_input_submit
    ]
)

### Load the model

- **Predictor variables X:** AGE07 EDCL07 MARRIED07 KIDS07 OCCAT107 WSAVED07 NETWORTH07 INCOME07 RISK07
- **Response variable   Y:** RTAvg

In [6]:
# Load the model
rt_model = pickle.load(open('../rfmodel_risk_tolerance_predictor.sav', 'rb'))

# Function to predict risk tolerance given x inputs
def predict_risk_tolerance(X_inputs, model=rt_model):
    return model.predict(X_inputs)


In [7]:
# Test with an example 

X_inputs1 = [
    np.array([40, 4, 1, 1, 2, 3, 2000000, 500000, 2])
]    
predict_risk_tolerance(X_inputs1)

array([0.43028782])

### Get stock data
- Stock tickers from a list of S&P500 companies on a Wikipedia page.
- Asset prices (adjusted) are retrieved from Yahoo Finance and saved in a static csv file.

In [8]:
def get_asset_data():
    assets = pd.read_csv('../data/prices.csv', index_col=0)
    total_tickers = len(assets.columns)
    print('Total number of companies: %s' %total_tickers)
    return assets

In [9]:
# Function to clean stock data

def final_asset_data(start_since='2000-01-01'):
    assets = get_asset_data()
    
    # Dealing with the historic price information from 2000 onwards
    print(f'Subsetting data to range from {start_since}')
    assets = assets.loc[start_since:].copy()
    
    
    # Dealing with missing values
    null_info = assets.isnull().mean(axis=0).sort_values(ascending=False) 
    drop_list = list(null_info[null_info > 0.3].index)
    assets = assets.drop(columns=drop_list)
    print('Deleted asset ticker(s) that have more than 30% missing data\n',
          'Now dealing with {} tickers'.format(len(assets.columns)))
    
    # Fill missing values with interpolation
    # As it is time series with a quite narros interval (every business data)
    # interpolating (forward with the last value available) makes sense
    assets = assets.fillna(method='ffill')
    
    
    # There are still some tickers with missing values 
    # keep them for a while and let's visualize it first

    final_null = assets.isnull().any().sum() 
    print('Stock prices have been forward filled\n',
          'There still are {} tickers that have missing values'.format(int(assets.isnull().any().sum())))

    return assets
    

In [10]:
assets = final_asset_data()

Total number of companies: 505
Subsetting data to range from 2000-01-01
Deleted asset ticker(s) that have more than 30% missing data
 Now dealing with 422 tickers
Stock prices have been forward filled
 There still are 54 tickers that have missing values


Retrieved from a source code provided on Github for [Machine Learning & Data Science Blueprints for Finance](https://github.com/tatsath/fin-ml/blob/master/Chapter%205%20-%20Sup.%20Learning%20-%20Regression%20and%20Time%20Series%20models/Case%20Study%203%20-%20Investor%20Risk%20Tolerance%20and%20Robo-advisors/Sample-Robo%20Advisor.ipynb)

In [11]:
# Function to return calculated allocation and estimated return
# Using convex optimzer

#Asset allocation given the Return, variance
def get_asset_allocation(riskTolerance, stock_tickers):
        
    #ipdb.set_trace()
    assets_selected = assets.loc[:, stock_tickers].dropna()
    n_tickers = len(assets_selected.columns)
    return_vec = np.array(assets_selected.pct_change().dropna()).reshape(n_tickers,-1)    
    returns = np.asmatrix(return_vec)
    mus = 1-riskTolerance
        
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(return_vec))
    pbar = opt.matrix(np.mean(return_vec, axis=1))
    # Create constraint matrices
    G = -opt.matrix(np.eye(n_tickers))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n_tickers ,1))
    A = opt.matrix(1.0, (1, n_tickers))
    b = opt.matrix(1.0)
    # Calculate efficient frontier weights using quadratic programming
    portfolios = solvers.qp(mus*S, -pbar, G, h, A, b)
    w=portfolios['x'].T
    print (w)
    Alloc =  pd.DataFrame(data = np.array(portfolios['x']),index = assets_selected.columns)

    # Calculate efficient frontier weights using quadratic programming
    portfolios = solvers.qp(mus*S, -pbar, G, h, A, b)
    returns_final=(np.array(assets_selected) * np.array(w))
    returns_sum = np.sum(returns_final,axis =1)
    returns_sum_pd = pd.DataFrame(returns_sum, index = assets_selected.index )
    returns_sum_pd = returns_sum_pd - returns_sum_pd.iloc[0,:] + 100   
    return Alloc,returns_sum_pd

### Create risk tolerance output component

In [12]:
# Components for outputs
# Showing risk tolerance and recommended asset allocation 

risk_tolerance_output = html.Div([
        html.Label('Your calculated risk tolerance is (scale of 100)...'),
        dcc.Input(id='risk-tolerance-output', disabled=True) # output will be used as an input for allocation component       
])

### Create chart components

In [13]:
# Get options for dropdown menu

ticker_options = []

for ticker in assets.columns:
    ticker_dict = dict()
    ticker_dict['label'] = ticker
    ticker_dict['value'] = ticker
    ticker_options.append(ticker_dict)

# Input/interactive components

allocation_input = dbc.FormGroup(
    [
        dbc.Label('Select the assets for the portfolio: ', html_for='dropdown'),
        dcc.Dropdown(
            id='ticker-symbols',
            options=ticker_options,
            value=['AAPL', 'MSFT', 'IBM', 'GOOGL'],
            multi=True
        ),
        dbc.Button(id='button-allocation', 
                   children='Display the suggested asset allocation',
                   color="primary", 
                   block=True)
    ]
) 


In [14]:
charts = html.Div(
    children=[
        html.Div(
            children=dcc.Graph(
                id='chart-allocation',
                config={'displayModeBar': False},
            )
        ),
        html.Div(
            children=dcc.Graph(
                id='chart-performance',
                config={'displayModeBar': False},
            )
        )
    ]

)

## Run dashboard server

In [15]:
# Create a container that
# includes all components

app.layout = dbc.Container(    
    className='mt-5 justify-contents-around',
    children=[
        dbc.Row(
            children=[
                dbc.Col(
                    [
                        dbc.Alert("Dashboard", color="success"),
                        form
                    ],
                    width=4
                ),
        
                dbc.Col(
                    [
                        dbc.Alert("Allocation", color="danger"),
                        risk_tolerance_output,
                        allocation_input,
                        charts
                    ],
                    width=8,
                )    
            ]
        )
        
    ]    
)


In [16]:
# Callback function  
# that takes all the inputs and computes the cluster and the risk tolerance

@app.callback(
    [
        # Ouputs
        Output('risk-tolerance-output', 'value')
    ],
    [
        # Inputs - 1 button + 9 features
        Input('button-risktolerance', 'n_clicks'),
        Input('age-input', 'value'),
        Input('education-input', 'value'),
        Input('married-input', 'value'),
        Input('kids-input', 'value'),
        Input('occupation-input', 'value'),
        Input('wsave-input', 'value'),
        Input('networth-input', 'value'),
        Input('income-input', 'value'),
        Input('risk-input', 'value')                
    ]    
)

def update_risk_tolerance(n_clicks, age, education, married, kids, occupation, wsave, nw, income, risk):
    
    risk_tolerance = 0
    if n_clicks != None:
        X_inputs = [np.array([age, education, married, kids, occupation, wsave, nw, income, risk])]
        risk_tolerance = predict_risk_tolerance(X_inputs)

    return [round(float(risk_tolerance*100), 2)]


# Call back to calculate and plot asset allocation and price graph
# that gets risk tolerance rate (0-100) and stock_ticker(s) as inputs

@app.callback(
    [
        # Ouputs
        Output('chart-allocation', 'figure'),
        Output('chart-performance', 'figure')
    ],
    [
        # Inputs
        Input('button-allocation', 'n_clicks'),
        Input('risk-tolerance-output', 'value'),
        
    ],
    [
        # State allows you to pass along extra values without firing the callbacks.
        State('ticker-symbols', 'value')
    ]
)
def display_asset_allocation(n_clicks, risk_tolerance, stock_tickers):
    
    allocated, returns = get_asset_allocation(risk_tolerance, stock_tickers)
    
    allocation_chart = {
        'data': [
            {
                'x': allocated.index,
                'y': allocated.iloc[:,0] * 100,
                'type':'bar',
                'hovertemplate': '$%{y:.2f}<extra></extra>',                
            }
        ],
        'layout': {
            'title': {
                'text': 'Suggested Asset Allocation',
                'x': 'Ticker symbols',
                'y': 'Percentage',
                'xanchor': 'left'
            },
            'xaxis': {'fixedrange': True},
            'yaxis': {
                'fixedrange': True,
                'tickprefix': '%'
            },
            'colorway': ['#17B897']  # chart color
        }
    }
    
    performance_chart = {
        'data': [
            {
                'x': returns.index,
                'y': returns.iloc[:,0],
                'type':'lines',
                'hovertemplate': '$%{y:.2f}<extra></extra>',
            }
        ],
        'layout': {
            'title': {
                'text': 'Portfolio trend (Base: 100)',
                'x': 'Date',
                'y': 'Portfolio value',
                'xanchor': 'left'
            },
            'xaxis': {'fixedrange': True},
            'yaxis': {'fixedrange': True},
            'colorway': ['#E75480'],
        }
    }
    
    return allocation_chart, performance_chart


# Run a dash board using the local server

# inline 
# if __name__ == '__main__':
#     app.run_server(mode='inline') 

# on browser 
if __name__ == '__main__':
    app.run_server() 

Dash app running on http://127.0.0.1:8050/
     pcost       dcost       gap    pres   dres
 0: -8.5402e-04 -1.0011e+00  1e+00  0e+00  3e+00
 1: -8.5409e-04 -1.1087e-02  1e-02  9e-17  3e-02
 2: -8.6121e-04 -1.1833e-03  3e-04  3e-17  8e-04
 3: -9.8536e-04 -1.0442e-03  6e-05  2e-16  9e-20
 4: -1.0112e-03 -1.0136e-03  2e-06  1e-16  1e-19
 5: -1.0132e-03 -1.0132e-03  3e-08  1e-16  2e-19
Optimal solution found.
[ 7.13e-05  6.38e-01  2.80e-05  3.62e-01]

     pcost       dcost       gap    pres   dres
 0: -8.5402e-04 -1.0011e+00  1e+00  0e+00  3e+00
 1: -8.5409e-04 -1.1087e-02  1e-02  9e-17  3e-02
 2: -8.6121e-04 -1.1833e-03  3e-04  3e-17  8e-04
 3: -9.8536e-04 -1.0442e-03  6e-05  2e-16  9e-20
 4: -1.0112e-03 -1.0136e-03  2e-06  1e-16  1e-19
 5: -1.0132e-03 -1.0132e-03  3e-08  1e-16  2e-19
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -8.5402e-04 -1.0011e+00  1e+00  0e+00  3e+00
 1: -8.5409e-04 -1.1087e-02  1e-02  9e-17  3e-02
 2: -8.6121e-04 -1.1833e-03  3e-04  